In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-04 09:42:35.681132


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-03-04T09:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-03-04 09:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-02-03 09:00:00+0000', tz='UTC')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

# def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
#     """
#     Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

#     Args:
#         from_date (datetime or str): The start date for the data batch.
#         to_date (datetime or str): The end date for the data batch.

#     Returns:
#         pd.DataFrame: A DataFrame containing the simulated production data.
#     """
#     # Convert string inputs to datetime if necessary
#     if isinstance(from_date, str):
#         from_date = datetime.fromisoformat(from_date)
#     if isinstance(to_date, str):
#         to_date = datetime.fromisoformat(to_date)

#     # Validate input dates
#     if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
#         raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
#     if from_date >= to_date:
#         raise ValueError("'from_date' must be earlier than 'to_date'.")

#     # Shift dates back by 52 weeks (1 year)
#     historical_from_date = from_date - timedelta(weeks=52)
#     historical_to_date = to_date - timedelta(weeks=52)

#     # Load and filter data for the historical period
#     rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
#     rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

#     if historical_to_date.month != historical_from_date.month:
#         rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
#         rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
#         # Combine the filtered data
#         rides = pd.concat([rides_from, rides_to], ignore_index=True)
#     else:
#         rides = rides_from
#     # Shift the data forward by 52 weeks to simulate recent data
#     rides['pickup_datetime'] += timedelta(weeks=52)

#     # Sort the data for consistency
#     rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

#     return rides



def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks and remove timezone info
    historical_from_date = (from_date - timedelta(weeks=52)).replace(tzinfo=None)
    historical_to_date = (to_date - timedelta(weeks=52)).replace(tzinfo=None)

    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from['pickup_datetime'] = rides_from['pickup_datetime'].dt.tz_localize(None)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = rides_to['pickup_datetime'].dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    rides['pickup_datetime'] += timedelta(weeks=52)
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
883918,2025-02-12 16:25:44,2
1190619,2025-02-15 16:56:40,2
4076,2025-02-03 09:11:09,3
18208,2025-02-03 12:14:43,3
52630,2025-02-03 18:07:44,3
...,...,...
2930027,2025-03-04 08:58:49,263
2929004,2025-03-04 08:59:17,263
2932765,2025-03-04 08:59:32,263
2974478,2025-03-04 08:59:48,263


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-03 09:00:00,2,0
1,2025-02-03 10:00:00,2,0
2,2025-02-03 11:00:00,2,0
3,2025-02-03 12:00:00,2,0
4,2025-02-03 13:00:00,2,0
...,...,...,...
174691,2025-03-04 04:00:00,263,8
174692,2025-03-04 05:00:00,263,18
174693,2025-03-04 06:00:00,263,89
174694,2025-03-04 07:00:00,263,130


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174696 non-null  datetime64[ns]
 1   pickup_location_id  174696 non-null  int16         
 2   rides               174696 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [15]:
from src.data_utils import convert_pickup_hour_to_ny
ts_data = convert_pickup_hour_to_ny(ts_data)

In [16]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                           
---  ------              --------------   -----                           
 0   pickup_hour         174696 non-null  datetime64[ns, America/New_York]
 1   pickup_location_id  174696 non-null  int16                           
 2   rides               174696 non-null  int16                           
dtypes: datetime64[ns, America/New_York](1), int16(2)
memory usage: 2.0 MB


In [17]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-03-04 04:42:57,654 INFO: Initializing external client
2025-03-04 04:42:57,656 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 04:42:59,644 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


In [18]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 174696/174696 | Elapsed Time: 00:26 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215665/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)